<a href="https://colab.research.google.com/github/DuanMingbai/PyTorch_Tutorials_cn/blob/main/Introduction%20to%20PyTorch/1_%E5%BF%AB%E9%80%9F%E5%BC%80%E5%A7%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 快速开始


本节贯穿机器学习常用任务的API，参阅每一个部分中的链接来深入了解

## 处理数据

PyTorch有两个处理数据的方法：```torch.utils.data.DataLoader```和```torch.utils.data.Dataset```。```dataset```存储了数据的样例和它的标签，```DataLoader```围绕着```dataset```打包了一个可迭代的对象。





In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch提供了特定领域的库例如TorchText，TorchVision和TorchAudio，所有的这些库都包含了他们自己的数据集。在本教程中，我们会使用TorchVision的数据集。

```torchvision.datasets```模块包含许多现实世界视觉数据的DataSet方法，像CIFAR，COCO([这里是全部列表](https://pytorch.org/vision/stable/datasets.html))。在本教程中，我们使用FasionMINST数据集。每一个TorchVision的```dataset```都包含了两个参数：```transform```和```target_transform```分别用来修改样本和标签。

In [ ]:
#从开源数据集下载训练集
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform = ToTensor(),
)
#从开源数据集下载测试集
test_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform = ToTensor(), 
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



我们将```Dateset```作为参数传递给```DataLoader```。DataLoader将我们传过去的对象打包成一个可迭代并且支持自动批处理，采样，洗牌和多进程处理的对象。下面我们定义了一个大小为64的batch size，意思就是每一个元素都会返回一批大小为64的特征和标签。

In [ ]:
batch_size = 64

#创建DataLoader
train_dataloader = DataLoader(train_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
  print(f"Shape of X [N ,C ,H ,W]:{X.shape}")
  print(f"Shape of y:{y.shape}")
  break

Shape of X [N ,C ,H ,W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64])


##创建模型

我们创建一个继承于nn.Model的类用来在PyTorch中创建一个神经网络，我们在```__init__```函数中定义神经网络的层数并指定数据如何在```forward```函数中通过网络。为了加快神经网络的运算速度，如果我们的GPU能用，我么就把神经网络的运算转移到GPU上来运算。

In [ ]:
from typing_extensions import Self
#获取训练硬件（CPU或者GPU）
device="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


阅读更多关于[在PyTorch中构建神经网络](https://https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

##优化模型参数

在训练模型的过程中，我们需要损失函(loss function)和优化器(optimizer)。

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在一个单独的训练循环中，模型会在训练集上做出预测（分batch送进模型）并且反向传播误差来调整模型的参数

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    #计算预测误差
    pred = model(X)
    loss = loss_fn(pred,y)

    #反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还要根据模型在测试集的表现来检查模型的性能，确保模型正在学习

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练的过程通过不断的迭代来进行（epoch）。在每一次的迭代中，模型会学到参数去让预测结果更好。我们在模型的每一次迭代中打印模型的准确度和损失，我们希望开到在每一个epoch中准确度（accuracy）增加而损失（loss）减少。

In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------
loss: 2.299721  [    0/60000]
loss: 2.284278  [ 6400/60000]
loss: 2.263869  [12800/60000]
loss: 2.259103  [19200/60000]
loss: 2.230743  [25600/60000]
loss: 2.204708  [32000/60000]
loss: 2.218012  [38400/60000]
loss: 2.179318  [44800/60000]
loss: 2.173181  [51200/60000]
loss: 2.139044  [57600/60000]
Test Error: 
 Accuracy: 42.1%, Avg loss: 2.131714 

Epoch 2
------------------------
loss: 2.146179  [    0/60000]
loss: 2.128604  [ 6400/60000]
loss: 2.073800  [12800/60000]
loss: 2.095151  [19200/60000]
loss: 2.020308  [25600/60000]
loss: 1.970331  [32000/60000]
loss: 1.997376  [38400/60000]
loss: 1.916751  [44800/60000]
loss: 1.916963  [51200/60000]
loss: 1.841493  [57600/60000]
Test Error: 
 Accuracy: 58.7%, Avg loss: 1.840193 

Epoch 3
------------------------
loss: 1.878675  [    0/60000]
loss: 1.837690  [ 6400/60000]
loss: 1.729619  [12800/60000]
loss: 1.774141  [19200/60000]
loss: 1.646592  [25600/60000]
loss: 1.616136  [32000/60000]
loss: 1.629828  [

阅读更多关于[训练你的模型](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)

##保存模型

加载模型的一个通用方式是序列化一个内部的状态字典（包括模型的参数）

In [ ]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


##加载模型

加载模型的过程包括重新创建模型结构并将状态字典(这个东西说实话我也不知道是啥，还需继续学习)加载到其中。

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>